<a href="https://colab.research.google.com/github/martianbandit/notebooks/blob/main/measure_the_tread_depth_of_tires.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

# Function to load images and preprocess data
def load_data(data_dir, image_height, image_width):
    images = []
    tread_depths = []

    for subdir, _, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.jpg'):
                # Read image
                image_path = os.path.join(subdir, file)
                image = cv2.imread(image_path)
                image = cv2.resize(image, (image_width, image_height))
                images.append(image)

                # Extract tread depth from filename or metadata
                tread_depth_str = ''.join(filter(str.isdigit, file))  # Extract only numerical characters
                tread_depth = float(tread_depth_str)  # Convert to float
                tread_depths.append(tread_depth)

    # Convert lists to numpy arrays
    images = np.array(images)
    tread_depths = np.array(tread_depths)

    return images, tread_depths

# Load data
data_dir = '/kaggle/input/data707'  # Replace with the path to your dataset directory
image_height, image_width = 100, 100  # Specify image dimensions
images, tread_depths = load_data(data_dir, image_height, image_width)

# Split data into training and validation sets
train_images, val_images, train_tread_depths, val_tread_depths = train_test_split(
    images, tread_depths, test_size=0.2, random_state=42)

# Define the model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Output layer for tread depth prediction
])

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',  # Use MSE loss for regression
              metrics=['mae'])  # Mean Absolute Error as evaluation metric

# Train the model
history = model.fit(train_images, train_tread_depths,
                    epochs=10,
                    validation_data=(val_images, val_tread_depths))

# Evaluate the model
val_loss, val_mae = model.evaluate(val_images, val_tread_depths)
print("Validation Loss:", val_loss)
print("Validation MAE:", val_mae)

# Save the model
model.save('tread_depth_model.h5')


In [ ]:
# Evaluate the model
val_loss, val_mae = model.evaluate(val_images, val_tread_depths)
print("Validation Loss:", val_loss)
print("Validation MAE:", val_mae)

# Calculate accuracy (lower MAE is better)
accuracy = 1 - (val_mae / np.mean(val_tread_depths))
print("Accuracy:", accuracy)
